
# Assignment 3

This is a template notebook for Assignment 3.


## Install dependencies and initialization

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

In [ ]:
!pwd # shows current directory
!ls  # shows all files in this directory
!nvidia-smi # shows the specs and the current status of the allocated GPU

In [ ]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os


# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch
import matplotlib.pyplot as plt

# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
setup_logger()

In [ ]:
# Make sure that GPU is available for your notebook. 
# Otherwise, you need to update the settungs in Runtime -> Change runtime type -> Hardware accelerator
torch.cuda.is_available()

In [ ]:
# You need to mount your google drive in order to load the data:
from google.colab import drive
drive.mount('/content/drive')
# Put all the corresponding data files in a data folder and put the data folder in a same directory with this notebook.
# Also create an output directory for your files such as the trained models and the output images.

In [ ]:
# Define the location of current directory, which should contain data/train, data/test, and data/train.json.
# TODO: approx 1 line
BASE_DIR = '/content/drive/My Drive/CMPT_CV_lab3'
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Part 1: Object Detection

### Data Loader

In [ ]:
'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# TODO: approx 35 lines
'''
# RUN THE improvement block first for data processing method !!!!

def get_detection_data(set_name):
  data_dirs = '{}/data'.format(BASE_DIR)
  dataset = []
  if set_name == "train":
    json_file = os.path.join(data_dirs, "train.json")
    data_dir = os.path.join(data_dirs, "train")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    img_id = -1
    record = {}
    objs = {}
    for idx, elem in enumerate(imgs_anns):
      if img_id != elem["image_id"]:
        if idx != 0:
          record["annotations"] = objs 
          dataset.append(record)
        record = {}       
        filename = os.path.join(data_dir, elem["file_name"])
        height, width = cv2.imread(filename).shape[:2]
        
        img_id = elem["image_id"]
        id = elem["id"]
        bbox = elem["bbox"]

        record["file_name"] = filename
        record["image_id"] = elem["image_id"]
        record["height"] = height
        record["width"] = width
      
        objs = []

        obj = {
            "bbox": bbox,
            "bbox_mode": BoxMode.XYWH_ABS,
            "segmentation": elem["segmentation"],
            "category_id": 0
        }
        objs.append(obj)
      else:
        obj = {
          "bbox": elem["bbox"],
          "bbox_mode": BoxMode.XYWH_ABS,
          "segmentation": elem["segmentation"],
          "category_id": 0
        }
        objs.append(obj)
    record["annotations"] = objs
    dataset.append(record)
  elif set_name == "test":
    data_dir = os.path.join(data_dirs, "test")
    for file in os.listdir(data_dir):
      record = {}       
      filename = os.path.join(data_dir, file)
      height, width = cv2.imread(filename).shape[:2]
      record["file_name"] = filename
      record["height"] = height
      record["width"] = width
      record["annotations"] = []
      dataset.append(record)
  return dataset

In [ ]:
'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consdier "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by spliting the training data
# TODO: approx 5 lines
'''
# RUN THE improvement block first for data processing method !!!!
for d in ["train", "test"]:
    DatasetCatalog.register("data_detection_" + d, lambda d=d: get_detection_data(d))
    MetadataCatalog.get("data_detection_" + d).set(thing_classes=["plane"])
plane_train_metadata = MetadataCatalog.get("data_detection_train")

#rigister the new block dataset
dataset_train_dicts = get_detection_data("train")

for d in ["train_block"]:
  DatasetCatalog.register("data_detection_" + d, lambda d=d: dataProcess(dataset_train_dicts))
  MetadataCatalog.get("data_detection_" + d).set(thing_classes=["plane"])
plane_train_block_metadata = MetadataCatalog.get("data_detection_train_block")

In [ ]:
'''
# Visualize some samples using Visualizer to make sure that the function works correctly
# TODO: approx 5 lines
'''

# RUN THE improvement block first for data processing method !!!!
dataset_train_block_dicts = dataProcess(dataset_train_dicts)

for d in [dataset_train_block_dicts[0],dataset_train_block_dicts[1], dataset_train_block_dicts[3]]:
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=plane_train_block_metadata)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])


### Set Configs

In [ ]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train_block",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 10
cfg.SOLVER.BASE_LR = 0.01  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.RETINANET.NUM_CLASSES = 1  
cfg.MODEL.RETINANET.IOU_THRESHOLDS = [0.4, 0.5]
cfg.MODEL.RETINANET.IOU_LABELS = [0, -1, 1]
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.




### Training

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()


In [ ]:
%load_ext tensorboard
%tensorboard --logdir='/content/drive/My Drive/CMPT_CV_lab3/output/'

### Evaluation and Visualization

In [ ]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''
dataset_test_dicts = get_detection_data("test")
plane_test_metadata = MetadataCatalog.get("data_detection_test")
for d in random.sample(dataset_test_dicts, 3):    
    b_img_list, d_list = get_block_imgs(d, "test")
    outputs_list = [] 
    out_image = np.zeros((d["height"],d["width"],3), np.uint8)
    # divide the test image into blocks
    b_w = d_list[0]["width"]
    b_h = d_list[0]["height"]
    num_col = 1
    num_row = 1
    if d["width"] >= 2000:
      num_col = d["width"]//1000
    if d["height"] >= 2000:
      num_row = d["height"]//1000

    for i in range(len(b_img_list)):
      pred = predictor(b_img_list[i])
      v = Visualizer(b_img_list[i][:, :, ::-1],
                    metadata=plane_test_metadata, 
                      # remove the colors of unsegmented pixels. This option is only available for segmentation models
      )
      out = v.draw_instance_predictions(pred["instances"].to("cpu"))
      sub_img = out.get_image()[:, :, ::-1]
      current_row = i%num_row
      current_col = i//num_row
      # merge the block images to whole image and visualize the result
      out_image[current_row*b_h:(current_row+1)*b_h, current_col*b_w:(current_col+1)*b_w] = sub_img
    cv2_imshow(out_image)


In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
evaluator = COCOEvaluator("data_detection_train_block", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "data_detection_train_block")
print(inference_on_dataset(predictor.model, val_loader, evaluator))


### Improvements

For this part, you can bring any improvement which you have by adding new input parameters to the previous functions or defining new functions and variables.

In [ ]:
'''
# Bring any changes and updates regarding the improvement in here
''' 
import math
## data processing
def get_block_imgs(elem, dir):
    block_img_list = []
    data_list = []
    filename = elem["file_name"]
    anno = elem["annotations"]
    width = elem["width"]
    height = elem["height"]
    img = cv2.imread(filename)
    n_hori_block = 1
    n_vert_block = 1
    if width > 2000:
      n_hori_block = width//1000
    if height > 2000:
      n_vert_block = height//1000
    new_w = math.floor(float(width)/n_hori_block)
    new_h = math.floor(float(height)/n_vert_block)
    for i in range(n_hori_block):
      for j in range(n_vert_block):
        block_img = img[j*new_h:(j+1)*new_h, i*new_w:(i+1)*new_w]
        block_img_list.append(block_img)
        data = {} 
        if dir!="test":
          data["file_name"] = os.path.join(dir, filename[-9:-4]+str(j)+str(i)+".png")
        data["width"] = new_w
        data["height"] = new_h
        new_anno_list = []
        if elem["annotations"] != []:
          objs = elem["annotations"]
          for obj in objs:
            if obj["bbox"][0]>i*new_w and obj["bbox"][0]<(i+1)*new_w and obj["bbox"][1]>j*new_h and obj["bbox"][1]<(j+1)*new_h :
              if obj["bbox"][0]+obj["bbox"][2] < (i+1)*new_w and obj["bbox"][1]+obj["bbox"][3] < (j+1)*new_h:
                new_ss = []
                for seg in obj["segmentation"]:
                  new_s=[]
                  for k in range(0,len(seg),2):
                    new_s.append(seg[k]-i*new_w)
                    new_s.append(seg[k+1]-j*new_h)
                  new_ss.append(new_s)
                new_anno = {
                    "bbox": [obj["bbox"][0]-i*new_w,obj["bbox"][1]-j*new_h,obj["bbox"][2],obj["bbox"][3]],
                    "bbox_mode":BoxMode.XYWH_ABS,
                    "segmentation": new_ss,
                    "category_id":0
                }
                new_anno_list.append(new_anno)
        data["annotations"] = new_anno_list
        data_list.append(data)
    return block_img_list, data_list


def dataProcess(dataset):
  block_dataset = []
  data_dirs = '{}/data'.format(BASE_DIR)
  block_data_dir = '{}/train_block'.format(data_dirs)
  os.makedirs(block_data_dir, exist_ok=True)
  for elem in dataset:
    block_img_list, data_list = get_block_imgs(elem, block_data_dir)
    # save the block image to new training folder
    # append the data_list to block_dataset, do not append it if no box in data
    num = 0
    for i in range(len(data_list)):
      if data_list[i]["annotations"] != []:
        cv2.imwrite(os.path.join(block_data_dir, data_list[i]["file_name"]), block_img_list[i])
        data_list[i]["image_id"] = num
        num+=1
        block_dataset.append(data_list[i])
  return block_dataset






## Part 2: Semantic Segmentation

### Data Loader

In [ ]:
'''
# Write a function that returns the cropped image and corresponding mask regarding the target bounding box
# idx is the index of the target bbox in the data
# high-resolution image could be passed or could be load from data['file_name']
# You can use the mask attribute of detectron2.utils.visualizer.GenericMask 
#     to convert the segmentation annotations to binary masks
# TODO: approx 10 lines
'''
from copy import deepcopy
def get_instance_sample(data, idx, img=None):
  if img is None:
    img = cv2.imread(data["file_name"])
  bbox = data["annotations"][idx]["bbox"]
  bbox[0] = int(bbox[0])
  bbox[1] = int(bbox[1])
  bbox[2] = int(bbox[2])
  bbox[3] = int(bbox[3])
  seg = deepcopy(data["annotations"][idx]["segmentation"])

  for i in range(0, len(seg), 1):  
    for j in range(0, len(seg[i]), 2):
      seg[i][j]-=bbox[0]
      seg[i][j+1]-=bbox[1]
  obj_img = img[bbox[1]:bbox[1]+bbox[3],bbox[0]:bbox[0]+bbox[2]]

  v_poly = detectron2.utils.visualizer.GenericMask(seg,bbox[3],bbox[2])
  obj_mask = v_poly.mask

  return obj_img, obj_mask


In [ ]:
'''
# We have provided a template data loader for your segmentation training
# You need to complete the __getitem__() function before running the code
# You may also need to add data augmentation or normalization in here
'''

class PlaneDataset(Dataset):
  def __init__(self, set_name, data_list):
      self.transforms = transforms.Compose([
          transforms.ToTensor(), # Converting the image to tensor and change the image format (Channels-Last => Channels-First)
      ])
      self.set_name = set_name
      self.data = data_list
      self.instance_map = []
      for i, d in enumerate(self.data):
        for j in range(len(d['annotations'])):
          self.instance_map.append([i,j])

  '''
  # you can change the value of length to a small number like 10 for debugging of your training procedure and overfeating
  # make sure to use the correct length for the final training
  '''
  def __len__(self):
      return len(self.instance_map)

  def numpy_to_tensor(self, img, mask):
    if self.transforms is not None:
        img = self.transforms(img)
    img = torch.tensor(img, dtype=torch.float)
    mask = torch.tensor(mask, dtype=torch.float)
    return img, mask

  '''
  # Complete this part by using get_instance_sample function
  # make sure to resize the img and mask to a fixed size (for example 128*128)
  # you can use "interpolate" function of pytorch or "numpy.resize"
  # TODO: 5 lines
  '''
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    idx = self.instance_map[idx]
    data = self.data[idx[0]]
    img, mask = get_instance_sample(data, idx[1])
    img = cv2.resize(img, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
    mask = cv2.resize(mask, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
    img, mask = self.numpy_to_tensor(img, mask)
    return img, mask

def get_plane_dataset(set_name='train', batch_size=2):
    my_data_list = DatasetCatalog.get("data_detection_{}".format(set_name))
    dataset = PlaneDataset(set_name, my_data_list)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=4,
                                              pin_memory=True, shuffle=True)
    return loader, dataset

### Network

In [ ]:
'''
# convolution module as a template layer consists of conv2d layer, batch normalization, and relu activation
'''
class conv(nn.Module):
    def __init__(self, in_ch, out_ch, activation=True):
        super(conv, self).__init__()
        if(activation):
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1),
             nn.BatchNorm2d(out_ch),
             nn.ReLU(inplace=True)
          )
        else:
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1)  
             )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# downsampling module equal to a conv module followed by a max-pool layer
'''
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.layer = nn.Sequential(
            conv(in_ch, out_ch),
            nn.MaxPool2d(2)
            )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# upsampling module equal to a upsample function followed by a conv module
'''
class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        if bilinear:
            self.layer = nn.Sequential(
                nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            )
        else:
            self.layer = nn.Sequential(
                nn.ConvTranspose2d(in_ch, out_ch, 3, padding=1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True)           
            )

        #self.conv = conv(in_ch, out_ch)

    def forward(self, x):
        y = self.layer(x)
        return y

'''
# the main model which you need to complete by using above modules.
# you can also modify the above modules in order to improve your results.
'''
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        # Encoder

        self.input_conv = conv(3, 64)
        self.down1 = down(64,64)
        self.conv1 = conv(64,128)
        self.down2 = down(128,128)
        self.conv2 = conv(128,256)
        self.conv3 = conv(256,256)
        self.down3 = down(256,256)
        self.conv4 = conv(256,512)
        self.conv5 = conv(512,512)
        self.down4 = down(512,512)
        
        # Decoder

        self.up1 = up(512,512,bilinear=True)
        self.up2 = up(512,512)
        self.up3 = up(512,256)
        self.up4 = up(256,256,bilinear =True)
        self.up5 = up(256,256)
        self.up6 = up(256,128)
        self.up7 = up(128,128,bilinear =True)
        self.up8 = up(128,64)
        self.up9 = up(64,64,bilinear =True)  

        self.output_conv = conv(64, 1, False) # ReLu activation is removed to keep the logits for the loss function


    def forward(self, input):
        y = self.input_conv(input)
        y = self.down1(y)
        y = self.conv1(y)
        y = self.down2(y)
        y = self.conv2(y)
        y = self.conv3(y)
        y = self.down3(y)
        y = self.conv4(y)
        y = self.conv5(y)
        y = self.down4(y)

        y = self.up1(y)
        y = self.up2(y)
        y = self.up3(y)
        y = self.up4(y)
        y = self.up5(y)
        y = self.up6(y)
        y = self.up7(y)
        y = self.up8(y)
        y = self.up9(y)
        output = self.output_conv(y)

        return output

### Training

In [ ]:
'''
# The following is a basic training procedure to train the network
# You need to update the code to get the best performance
# TODO: approx ? lines
'''
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice

class IoULoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(IoULoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #intersection is equivalent to True Positive count
        #union is the mutually inclusive area of all labels & predictions 
        intersection = (inputs * targets).sum()
        total = (inputs + targets).sum()
        union = total - intersection 
        
        IoU = (intersection + smooth)/(union + smooth)
                
        return 1 - IoU

# Set the hyperparameters
num_epochs = 20
batch_size = 100
learning_rate = 0.02
weight_decay = 1e-5

model = MyModel() # initialize the model
model = model.cuda() # move the model to GPU
loader, _ = get_plane_dataset('train', batch_size) # initialize data_loader
crit1 = nn.BCEWithLogitsLoss() # Define the loss function
crit2 = DiceLoss()
crit3 = IoULoss()
optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Initialize the optimizer as SGD
plt.ioff()
fig = plt.figure()
train_loss_over_epochs=[]

# start the training procedure
for epoch in range(num_epochs):
  total_loss = 0
  for (img, mask) in tqdm(loader):
    img = torch.tensor(img, device=torch.device('cuda'), requires_grad = True)
    mask = torch.tensor(mask, device=torch.device('cuda'), requires_grad = True)
    mask = torch.unsqueeze(mask, 1)
    pred = model(img)
    loss = crit1(pred,mask)+crit2(pred,mask)+crit3(pred,mask)
    optim.zero_grad()
    loss.backward()
    optim.step()
    total_loss += loss.cpu().data
  print("Epoch: {}, Loss: {}".format(epoch, total_loss/len(loader)))
  torch.save(model.state_dict(), '{}/output/{}_segmentation_model.pth'.format(BASE_DIR, epoch))
  train_loss_over_epochs.append(total_loss)

# plot the figure
plt.subplot(2, 1, 1)
plt.ylabel('Train loss')
plt.plot(np.arange(num_epochs), train_loss_over_epochs, 'k-')
plt.title('train loss')
plt.xticks(np.arange(num_epochs, dtype=int))
plt.grid(True)
plt.savefig(os.path.join(BASE_DIR,"plot.png"))
plt.close(fig)

'''
# Saving the final model
'''
torch.save(model.state_dict(), '{}/output/final_segmentation_model.pth'.format(BASE_DIR))


### Evaluation and Visualization

In [ ]:
'''
# Before starting the evaluation, you need to set the model mode to eval
# You may load the trained model again, in case if you want to continue your code later
# TODO: approx 15 lines
'''
batch_size = 100
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # chaning the model to evaluation mode will fix the bachnorm layers
loader, dataset = get_plane_dataset('train', batch_size)
total_iou = 0
for (img, mask) in tqdm(loader):
  with torch.no_grad():
    img = img.cuda()
    mask = mask.cuda()
    mask = torch.unsqueeze(mask,1)
    pred = model(img)

    '''
    ## Complete the code by obtaining the IoU for each img and print the final Mean IoU
    '''
    # set the threshold to 0.5
    pred[pred>=0.5] = 1
    pred[pred<0.5] = 0
    for i in range(0,len(mask)):
      intersection = torch.sum(torch.mul(mask[i],pred[i]))
      union = torch.sum(mask[i])+torch.sum(pred[i])-intersection
      IoU = intersection/union
      total_iou+=IoU
   
num_img = dataset.__len__()
print("\n #images: {}, Mean IoU: {}".format(num_img, total_iou/num_img))


In [ ]:
'''
# Visualize 3 sample outputs
# TODO: approx 5 lines
'''

# reload the test dataset if needed
#dataset_test_dicts = get_detection_data("test") 
def drawSampleAndResult(img):
  cv2_imshow(img)

  # mask1 = cv2.resize(mask1, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
  # cv2_imshow(mask1*255)
  img = np.moveaxis(img,2,0)
  img = img/255

  img = torch.tensor(img, device=torch.device('cuda'), requires_grad = False, dtype=torch.float)
  img = torch.unsqueeze(img,0)
  img = img.cuda()

  pred1 = model(img)
  out = pred1.data
  out = torch.squeeze(out)
  out = out.cpu().numpy()
  out[out>=0.5] = 1
  out[out<0.5] = 0
  cv2_imshow(out*255)

img = cv2.imread(dataset_test_dicts[1]["file_name"])
instances = predictor(img)["instances"].to("cpu")
img_pred_boxes = instances.get("pred_boxes")
bbox = img_pred_boxes[0].tensor.numpy().squeeze()
bbox = [round(bbox[0]),round(bbox[1]),round(bbox[2]),round(bbox[3])]
img1 = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
img1 = cv2.resize(img1, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
drawSampleAndResult(img1)

img = cv2.imread(dataset_test_dicts[2]["file_name"])
instances = predictor(img)["instances"].to("cpu")
img_pred_boxes = instances.get("pred_boxes")
bbox = img_pred_boxes[0].tensor.numpy().squeeze()
bbox = [round(bbox[0]),round(bbox[1]),round(bbox[2]),round(bbox[3])]
img1 = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
img1 = cv2.resize(img1, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
drawSampleAndResult(img1)

img = cv2.imread(dataset_test_dicts[4]["file_name"])
instances = predictor(img)["instances"].to("cpu")
img_pred_boxes = instances.get("pred_boxes")
bbox = img_pred_boxes[0].tensor.numpy().squeeze()
bbox = [round(bbox[0]),round(bbox[1]),round(bbox[2]),round(bbox[3])]
img1 = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
img1 = cv2.resize(img1, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
drawSampleAndResult(img1)

# img2, mask2 = get_instance_sample(dataset_train_dicts[0],15)
# cv2_imshow(img2)
# cv2_imshow(mask2*255)
# pred2 = model(img2)
# pred2[pred2>=0.5] = 1
# pred2[pred2<0.5] = 0
# cv2_imshow(pred2*255)

# img3, mask3 = get_instance_sample(dataset_train_dicts[0],20)
# cv2_imshow(img3)
# cv2_imshow(mask3*255)
# pred3 = model(img3)
# pred3[pred3>=0.5] = 1
# pred3[pred3<0.5] = 0
# cv2_imshow(pred3*255)

## Part 3: Instance Segmentation

In this part, you need to obtain the instance segmentation results for the test data by using the trained segmentation model in the previous part and the detection model in Part 1.

### Get Prediction

In [ ]:
'''
# Define a new function to obtain the prediction mask by passing a sample data
# For this part, you need to use all the previous parts (predictor, get_instance_sample, data preprocessings, etc)
# It is better to keep everything (as well as the output of this funcion) on gpu as tensors to speed up the operations.
# pred_mask is the instance segmentation result and should have different values for different planes.
# TODO: approx 35 lines
'''

def get_prediction_mask(data):
  img = cv2.imread(data["file_name"])
  objs = data["annotations"]
  w = data["width"]
  h = data["height"]
  gt_mask = np.zeros((h, w))
  pred_mask = np.zeros((h,w))
  for idx in range (len(objs)):
    im, mask = get_instance_sample(data,idx,img)
    bbox = objs[idx]["bbox"]
    gt_mask[bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]] = mask

  b_img_list, d_list = get_block_imgs(data, "test")
  outputs_list = [] 
  b_w = d_list[0]["width"]
  b_h = d_list[0]["height"]
  num_col = 1
  num_row = 1
  if data["width"] >= 2000:
    num_col = data["width"]//1000
  if data["height"] >= 2000:
    num_row = data["height"]//1000
  num_instances = 0
  for i in range(len(b_img_list)):
    current_row = i%num_row
    current_col = i//num_row
    sub_pred_mask = np.zeros((b_h,b_w))
    detection_pred = predictor(b_img_list[i])["instances"].to("cpu")
    bboxes_detected = detection_pred.get("pred_boxes")
    for j in range(len(bboxes_detected)):
      num_instances+=1
      bbox_pred =  bboxes_detected[j].tensor.numpy().squeeze()
      bbox_pred = [round(bbox_pred[0]),round(bbox_pred[1]),round(bbox_pred[2]),round(bbox_pred[3])]
      box_img = b_img_list[i][bbox_pred[1]:bbox_pred[3], bbox_pred[0]:bbox_pred[2]]
      #cv2_imshow(box_img)

      box_img = cv2.resize(box_img, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
      box_img = np.moveaxis(box_img,2,0)
      box_img = box_img/255

      box_img = torch.tensor(box_img, device=torch.device('cuda'), requires_grad = False, dtype=torch.float)
      box_img = torch.unsqueeze(box_img,0)
      box_img = box_img.cuda()

      pred1 = model(box_img)
      out = pred1.data
      out = torch.squeeze(out)
      out = out.cpu().numpy()
      out[out>=0.5] = num_instances
      out[out<0.5] = 0
      mask = cv2.resize(out, dsize=(bbox_pred[2]-bbox_pred[0], bbox_pred[3]-bbox_pred[1]), interpolation=cv2.INTER_NEAREST)
      sub_pred_mask[bbox_pred[1]:bbox_pred[3], bbox_pred[0]:bbox_pred[2]] = mask
    pred_mask[current_row*b_h:(current_row+1)*b_h, current_col*b_w:(current_col+1)*b_w] = sub_pred_mask
  pred_mask = torch.tensor(pred_mask, device=torch.device('cuda'), dtype=torch.long)
  return img, gt_mask, pred_mask # gt_mask could be all zero when the ground truth is not given.


### Visualization and Submission

In [ ]:
'''
# Visualise the output prediction as well as the GT Mask and Input image for a sample input
# TODO: approx 10 lines
'''
c=0
for s in [0, 10, 30]:
  c+=1
  img1, gt_mask1, pred_mask1 = get_prediction_mask(dataset_test_dicts[s])

  pred_mask1 = pred_mask1.cpu().numpy()
  for i in range(pred_mask1.shape[0]):
    for j in range(pred_mask1.shape[1]):
      if(pred_mask1[i][j] > 0):
        pred_mask1[i][j] = 255 - (pred_mask1[i][j]-1)*2
  print("result ", c," for test data")
  print("image:")
  img1 = cv2.resize(img1, (0,0), fx=0.2, fy=0.2) 
  pred_mask1 = pred_mask1.astype('float32')
  pred_mask1 = cv2.resize(pred_mask1, (0,0), fx=0.2, fy=0.2) 
  cv2_imshow(img1)
  print("pred_mask:")
  cv2_imshow(pred_mask1)



In [ ]:
'''
# ref: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# https://www.kaggle.com/c/airbus-ship-detection/overview/evaluation
'''
def rle_encoding(x):
    '''
    x: pytorch tensor on gpu, 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = torch.where(torch.flatten(x.long())==1)[0]
    if(len(dots)==0):
      return []
    inds = torch.where(dots[1:]!=dots[:-1]+1)[0]+1
    inds = torch.cat((torch.tensor([0], device=torch.device('cuda'), dtype=torch.long), inds))
    tmpdots = dots[inds]
    inds = torch.cat((inds, torch.tensor([len(dots)], device=torch.device('cuda'))))
    inds = inds[1:] - inds[:-1]
    runs = torch.cat((tmpdots, inds)).reshape((2,-1))
    runs = torch.flatten(torch.transpose(runs, 0, 1)).cpu().data.numpy()
    return ' '.join([str(i) for i in runs])

In [ ]:
'''
# You need to upload the csv file on kaggle
# The speed of your code in the previous parts highly affects the running time of this part
'''

preddic = {"ImageId": [], "EncodedPixels": []}

'''
# Writing the predictions of the training set
'''
my_data_list = DatasetCatalog.get("data_detection_{}".format('train'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

'''
# Writing the predictions of the test set
'''

my_data_list = DatasetCatalog.get("data_detection_{}".format('test'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for j, index in enumerate(inds):
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index).double()
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

pred_file = open("{}/pred.csv".format(BASE_DIR), 'w')
pd.DataFrame(preddic).to_csv(pred_file, index=False)
pred_file.close()


## Part 4: Mask R-CNN

For this part you need to follow a same procedure to part 2 with the configs of Mask R-CNN, other parts are generally the same as part 2.

### Data Loader

In [ ]:
#dataset_train_block_dicts = dataProcess(dataset_train_dicts)
for i in range(len(dataset_train_block_dicts)):
  _, gt_masks, _ = get_prediction_mask(dataset_train_block_dicts[i])
  dataset_train_block_dicts[i]["gt_masks"] = gt_masks



### Network

In [ ]:
cfg2 = get_cfg()
cfg2.OUTPUT_DIR = "{}/output2/".format(BASE_DIR)
cfg2.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg2.DATASETS.TRAIN = ("data_detection_train_block",)
cfg2.DATASETS.TEST = ()
cfg2.DATALOADER.NUM_WORKERS = 2
cfg2.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg2.SOLVER.IMS_PER_BATCH = 8
cfg2.SOLVER.BASE_LR = 0.01  # pick a good LR
cfg2.SOLVER.MAX_ITER = 200    
cfg2.SOLVER.STEPS = []        # do not decay learning rate
cfg2.MODEL.ROI_HEADS.NUM_CLASSES = 1


### Training

In [ ]:
os.makedirs(cfg2.OUTPUT_DIR, exist_ok=True)
trainer2 = DefaultTrainer(cfg2) 
trainer2.resume_or_load(resume=False)
trainer2.train()


In [ ]:
# load the weight and generate predictor here

cfg2.MODEL.WEIGHTS = os.path.join(cfg2.OUTPUT_DIR, "model_final.pth")
predictor2 = DefaultPredictor(cfg2)

### Evaluation and Visualization

In [ ]:
c = 0
for d in [dataset_test_dicts[0], dataset_test_dicts[10], dataset_test_dicts[30]]:    
    c+=1
    b_img_list, d_list = get_block_imgs(d, "test")
    outputs_list = [] 
    out_image = np.zeros((d["height"],d["width"]))
    # divide the test image into blocks
    b_w = d_list[0]["width"]
    b_h = d_list[0]["height"]
    num_col = 1
    num_row = 1
    if d["width"] >= 2000:
      num_col = d["width"]//1000
    if d["height"] >= 2000:
      num_row = d["height"]//1000

    for i in range(len(b_img_list)):
      pred = predictor2(b_img_list[i])
      v = Visualizer(b_img_list[i][:, :, ::-1],
                    metadata=plane_train_metadata, 
                      # remove the colors of unsegmented pixels. This option is only available for segmentation models
      )
      out = pred["instances"].to("cpu").get("pred_masks").numpy()
      out[out>0.5]=1
      fout = np.zeros((b_h,b_w))
      for j in range(len(out)):
        fout = fout+out[j]
      current_row = i%num_row
      current_col = i//num_row
      # merge the block images to whole image and visualize the result
      out_image[current_row*b_h:(current_row+1)*b_h, current_col*b_w:(current_col+1)*b_w] = fout
    print("result ",c)
    out_image = cv2.resize(out_image, (0,0), fx=0.2, fy=0.2) 
    cv2_imshow(out_image*255)

In [ ]:
evaluator = COCOEvaluator("data_detection_train_block", output_dir="./output")
val_loader2 = build_detection_test_loader(cfg2, "data_detection_train_block")
print(inference_on_dataset(predictor2.model, val_loader2, evaluator))
